In [1]:
from qulacs import QuantumCircuit, QuantumCircuitSimulator, ParametricQuantumCircuit, QuantumState
import numpy as np
import networkx as nx
from collections_extended import CountsView
from collections import Counter
from time import process_time

In [2]:
from tqdm import tqdm
from qulacs_core.gate import ParametricPauliRotation

In [3]:
int_to_binstr = lambda x, len :  bin(x)[2:].zfill(len)

In [3]:
##  RE-DEFINE SUBROUTINES  ##

def get_tsp_init_circuit(G: nx.Graph , init_state=None, encoding="onehot"):
    """
    Generates an inti state.

    Parameters
    ----------
    G : networkx.Graph
        Graph to solve TSP on
    init_state : list of integers

    Returns
    -------
    qc : ParametricQuantumCircuit
        Quantum circuit implementing the TSP phase unitary
    """
    if encoding == "onehot" and init_state:
        N = G.number_of_nodes()
        assert N == len(init_state)
        qc = ParametricQuantumCircuit(N**2)
        for i in range(N):
            qc.add_X_gate(i*N + init_state[i])
        return qc

    elif encoding == "onehot":
        N = G.number_of_nodes()
        qc = ParametricQuantumCircuit(N**2)
        for i in range(N):
            qc.add_X_gate(i*N+i)
        return qc

In [4]:
def _update_ordering_swap_partial_mixing_circuit(
    qstate, G, i, j, u, v, beta, T, encoding="onehot", structure="pauli rotations"):
    """
    Generates an ordering swap partial mixer for the TSP mixing unitary.

    Parameters
    ----------
    qstate : qulacs.QuantumState
             current quantum state
    G : networkx.Graph
        Graph to solve TSP on
    i, j :
        Positions in the ordering to be swapped
    u, v :
        Cities to be swapped
    beta :
        QAOA angle
    T :
        Number of Trotter steps
    encoding : string, default "onehot"
        Type of encoding for the city ordering

    Returns
    -------
    qstate
    """
    if encoding == "onehot" and structure == "pauli rotations":
        N = G.number_of_nodes()
        dt = beta/T
        # qc = ParametricQuantumCircuit(N**2)
        qui = (N*i + u)
        qvj = (N*j + v)
        quj = (N*j + u)
        qvi = (N*i + v)
        for t in range(T):
            # append_4_qubit_pauli_rotation_term(qc, qui, qvj, quj, qvi, dt, "xxxx")
            # qc.add_parametric_multi_Pauli_rotation_gate([qui, qvj, quj, qvi], [1,1,1,1], dt)
            ParametricPauliRotation([qui, qvj, quj, qvi], [1,1,1,1], dt).update_quantum_state(qstate)
            # append_4_qubit_pauli_rotation_term(qc, qui, qvj, quj, qvi, -dt, "xxyy")
            # qc.add_parametric_multi_Pauli_rotation_gate([qui, qvj, quj, qvi], [1,1,2,2], -dt)
            ParametricPauliRotation([qui, qvj, quj, qvi], [1,1,2,2], -dt).update_quantum_state(qstate)
            # append_4_qubit_pauli_rotation_term(qc, qui, qvj, quj, qvi, dt, "xyxy")
            # qc.add_parametric_multi_Pauli_rotation_gate([qui, qvj, quj, qvi], [1,2,1,2], dt)
            ParametricPauliRotation([qui, qvj, quj, qvi], [1,2,1,2], dt).update_quantum_state(qstate)
            # append_4_qubit_pauli_rotation_term(qc, qui, qvj, quj, qvi, dt, "xyyx")
            # qc.add_parametric_multi_Pauli_rotation_gate([qui, qvj, quj, qvi], [1,2,2,1], dt)
            ParametricPauliRotation([qui, qvj, quj, qvi], [1,2,2,1], dt).update_quantum_state(qstate)
            # append_4_qubit_pauli_rotation_term(qc, qui, qvj, quj, qvi, dt, "yxxy")
            # qc.add_parametric_multi_Pauli_rotation_gate([qui, qvj, quj, qvi], [2,1,1,2], dt)
            ParametricPauliRotation([qui, qvj, quj, qvi], [2,1,1,2], dt).update_quantum_state(qstate)
            # append_4_qubit_pauli_rotation_term(qc, qui, qvj, quj, qvi, dt, "yxyx")
            # qc.add_parametric_multi_Pauli_rotation_gate([qui, qvj, quj, qvi], [2,1,2,1], dt)
            ParametricPauliRotation([qui, qvj, quj, qvi], [2,1,2,1], dt).update_quantum_state(qstate)
            # append_4_qubit_pauli_rotation_term(qc, qui, qvj, quj, qvi, -dt, "yyxx")
            # qc.add_parametric_multi_Pauli_rotation_gate([qui, qvj, quj, qvi], [2,2,1,1], -dt)
            ParametricPauliRotation([qui, qvj, quj, qvi], [2,2,1,1], -dt).update_quantum_state(qstate)
            # append_4_qubit_pauli_rotation_term(qc, qui, qvj, quj, qvi, dt, "yyyy")
            # qc.add_parametric_multi_Pauli_rotation_gate([qui, qvj, quj, qvi], [2,2,2,2], dt)
            ParametricPauliRotation([qui, qvj, quj, qvi], [2,2,2,2], dt).update_quantum_state(qstate)
        return None

def _update_simultaneous_ordering_swap_mixer(qstate, G, beta, T1, T2, encoding="onehot"):
    if encoding == "onehot":
        N = G.number_of_nodes()
        dt = beta/T2
        # qc = ParametricQuantumCircuit(N**2)
        for t in range(T2):
            for i in range(N-1):
                for u, v in G.edges:
                    _update_ordering_swap_partial_mixing_circuit(qstate,
                                G, i, i+1, u, v, dt, T1, encoding="onehot")
            for u, v in G.edges:
                _update_ordering_swap_partial_mixing_circuit(qstate,
                                G, N-1, 0, u, v, dt, T1, encoding="onehot")
        return None

In [5]:
def _update_tsp_cost_operator_circuit(qstate,
    G, gamma, pen, encoding="onehot", structure= "zz rotation"):
    """
    Generates a circuit for the TSP phase unitary with optional penalty.

    Parameters
    ----------
    qstate : qulacs.QuantumState
             current quantum state
    G : networkx.Graph
        Graph to solve TSP on
    gamma :
        QAOA parameter gamma
    pen :
        Penalty for edges with no roads
    encoding : string, default "onehot"
        Type of encoding for the city ordering
    translate :
        dictionary with city encoding (ascending numerical to problem encoding)

    Returns
    -------
    qc : qiskit.QuantumCircuit
        Quantum circuit implementing the TSP phase unitary
    """
    if encoding == "onehot" and structure == "zz rotation":
        N = G.number_of_nodes()
        if not nx.is_weighted(G):
            raise ValueError("Provided graph is not weighted")
        # qc = ParametricQuantumCircuit(N**2)
        for n in range(N): # cycle over all cities in the input ordering
            for u in range(N):
                for v in range(N): #road from city v to city u
                    q1 = (n*N + u) % (N**2)
                    q2 = ((n+1)*N + v) % (N**2)
                    if G.has_edge(u, v):
                        # append_zz_term(qc, q1, q2, gamma * G[u][v]["weight"])
                        ParametricPauliRotation([q1,q2], [3,3], gamma * G[u][v]["weight"]).update_quantum_state(qstate)
                    else:
                        # append_zz_term(qc, q1, q2, gamma * pen)
                        ParametricPauliRotation([q1,q2], [3,3], gamma * pen).update_quantum_state(qstate)
        return None
    # if encoding == "onehot" and structure == "controlled z":
    #     N = G.number_of_nodes()
    #     if not nx.is_weighted(G):
    #         raise ValueError("Provided graph is not weighted")
    #     qc = QuantumCircuit(N**2)
    #     for n in range(N): # cycle over all cities in the input ordering
    #         for u in range(N):
    #             for v in range(N): #road from city v to city u
    #                 q1 = (n*N + u) % (N**2)
    #                 q2 = ((n+1)*N + v) % (N**2)
    #                 if G.has_edge(u, v):
    #                     qc.crz(gamma * G[u][v]["weight"], q1, q2)
    #                 else:
    #                     qc.crz(gamma * pen, q1, q2)
    #     return qc


In [6]:
def _update_tsp_qaoa_circuit(qstate,
    G, beta, gamma, T1=5, T2=5, pen=2,
    transpile_to_basis=True, save_state=True, encoding="onehot"
):
    if encoding == "onehot":
        
        assert len(beta) == len(gamma)
        p = len(beta)  # infering number of QAOA steps from the parameters passed
        N = G.number_of_nodes()
    
        
        # prepare the init state in onehot encoding
        get_tsp_init_circuit(G, encoding="onehot").update_quantum_state(qstate)

        # second, apply p alternating operators
        for i in range(p):
            _update_tsp_cost_operator_circuit(qstate, G, gamma[i], pen, encoding="onehot")
            _update_simultaneous_ordering_swap_mixer(qstate, G, beta[i], T1, T2, encoding="onehot")
            
        # if transpile_to_basis:
        #     qc = transpile(qc, optimization_level=0, basis_gates=["u1", "u2", "u3", "cx"])
        # if save_state:
        #     qc.save_state()

        return None





In [39]:
n = 5
G = nx.complete_graph(n)
for (u,v,w) in G.edges(data=True):
    w['weight'] = np.random.randn()

qstate = QuantumState(n**2)


In [40]:
tsp_qstate = _update_tsp_qaoa_circuit(qstate,G, [0.1], [0.1])
get_sample_counts(qstate)

{'1000001000001000001000001': 450,
 '1000001000000100010000001': 99,
 '101000001000001010000': 93,
 '1000000100010000001000001': 86,
 '100010000001000001000001': 89,
 '101000000100010010000': 12,
 '1001000000010010010000': 1,
 '1000000010010000010000001': 3,
 '100010010000010010000': 2,
 '1000001000001000000100010': 76,
 '100010000001000000100010': 14,
 '100100010000001010000': 14,
 '100010000000100010000001': 13,
 '10010000000100100000001': 3,
 '10010000010000000100010': 2,
 '1000000100010000000100010': 19,
 '100010000000010010000010': 2,
 '1000000100000100100000001': 3,
 '10010000010000001000001': 7,
 '110000001000001001000': 4,
 '100000010100000010000001': 1,
 '1000001000000010010000010': 8,
 '100000001001000001010000': 3,
 '101000001001000000010': 3,
 '10001000000100000110000': 1,
 '1001000001000000110000': 3,
 '100000100100000000100010': 2,
 '10000001010000001010000': 1,
 '1000001000000100000100100': 4,
 '1010000001000100000001': 1,
 '110000000100010001000': 2,
 '10001000100000001

In [8]:
n = tsp.get_qubit_count()
state = QuantumState(n)

In [9]:
# print(state)

In [10]:
tsp.update_quantum_state(state)
# print(state);

In [37]:
def get_sample_counts(qstate: QuantumState, shots:int= 1024):
    return dict([ (int_to_binstr(item, n) ,counts)  for item, counts in Counter(qstate.sampling(shots)).items()])



In [11]:
s = state.sampling(1024)
counts = dict([ (int_to_binstr(item, n) ,counts)  for item, counts in Counter(s).items()])
counts

{'1000001000001000001000001': 457,
 '0000101000001000001010000': 87,
 '0000100100010000001010000': 13,
 '0100010000001000001000001': 79,
 '1000001000000100010000001': 80,
 '0000101000000100010010000': 20,
 '0100010000000100010000001': 14,
 '1000001000001000000100010': 100,
 '1000000100010000000100010': 10,
 '1000000100010000001000001': 91,
 '1000000010010000010000001': 4,
 '0000110000001000001001000': 6,
 '0100010000001000000100010': 21,
 '0000100010010000010010000': 2,
 '0100010000000010010000010': 2,
 '0001000100010000000110000': 2,
 '0000110000000100010001000': 1,
 '0100000100100000001000001': 4,
 '1000001000000100000100100': 1,
 '1000001000000010010000010': 6,
 '0010010000010000001000001': 6,
 '0001010000010000010000001': 1,
 '0100000001001000001010000': 1,
 '0001001000001000000110000': 5,
 '0001010000001000000101000': 1,
 '0001001000000010010010000': 1,
 '1000000100000010100000010': 1,
 '1000000010001000100000001': 1,
 '0001001000100000010000001': 1,
 '1000000010010000000100100': 

In [12]:
from tspqaoa.qaoa import compute_tsp_cost_expectation

In [13]:
def get_tsp_expectation_value_method(G, pen, i_n=[]):
    
    """
    Runs parametrized circuit
    
    Args:
        G: networkx graph
        pen: int
            penalty for wrong formatted paths
        init_state: string
            initial state in the onehot encoding
    
    Returns :
        execute_circ method
    """
    
    #backend = Aer.get_backend('qasm_simulator')
    # aersim = AerSimulator(device=device)
    
    def execute_circ(angles):
        n = len(angles)
        assert n%2 == 0
        beta = angles[0:int(n/2)]
        gamma = angles[int(n/2):n]
        
        qc = get_tsp_qaoa_circuit(G, beta, gamma)
        
        qstate = QuantumState( qc.get_qubit_count() )
        qc.update_quantum_state(qstate)
        
        ## sample from state ##
        num_samples = 1024
        samples = qstate.sampling(num_samples)
        counts = dict([ (int_to_binstr(item, n) ,counts)  for item, counts in Counter(samples).items()])

        return compute_tsp_cost_expectation(counts, G, pen, i_n)
    
    return execute_circ

In [14]:
tsp_exp  = get_tsp_expectation_value_method(G, 0.004, [3] )

In [9]:
### scaling tests ###

n = 25
qc = ParametricQuantumCircuit(n)

for _ in range(10000):
    i = np.random.randint(0,int(n/4)); j = np.random.randint(int(n/4)+1, int(2*n/4)) % n
    k = np.random.randint(int(2*n/4)+1, 3*n/4) % n ; l = np.random.randint(int(3*n/4)+1, n) % n  
    
    qc.add_parametric_multi_Pauli_rotation_gate([i,j,k,l],[1,1,1,1], np.random.uniform(low=0,high= 2*np.pi) )


qs = QuantumState(n)
ti = process_time()
qc.update_quantum_state(qs)
ti = process_time() - ti
print('time ', ti)

time  8892.005384724


In [10]:
n = 25
# qc = ParametricQuantumCircuit(n)

ti = process_time()
qss = QuantumState(n)
for _ in tqdm(range(10000)):
    
    i = np.random.randint(0,int(n/4)); j = np.random.randint(int(n/4)+1, int(2*n/4)) % n
    k = np.random.randint(int(2*n/4)+1, 3*n/4) % n ; l = np.random.randint(int(3*n/4)+1, n) % n  
    
    ParametricPauliRotation([i,j,k,l],[1,1,1,1],np.random.uniform(low=0,high= 2*np.pi)).update_quantum_state(qss)
        


print(' time ', process_time() - ti )

 34%|███▍      | 3426/10000 [08:29<16:17,  6.72it/s]


KeyboardInterrupt: 

In [44]:
s = ParametricPauliRotation([1],[1], 1.0)

In [45]:
s.update_quantum_state(qs)

In [49]:
g = qc.get_gate(4)

In [50]:
g.update_quantum_state(qs)